In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 856.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import add, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input  # Or any other preprocessing based on your model
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
%%writefile app.py
import streamlit as st
import os
import numpy as np
import librosa
import cv2
import tempfile
from tensorflow.keras.models import load_model
from datetime import datetime
from PIL import Image
import time

# Load models
cnn_model = load_model(r"InceptionV3Model.h5")
lstm_model = load_model(r"AudioLSTMModel (1).h5")

# Constants
FRAME_SIZE = (256, 256)
MAX_FRAMES = 30
MFCC_FEATURES = 13
LSTM_INPUT_LENGTH = 130
THRESHOLD = 0.5

# Navigation
st.set_page_config(page_title="Multimodal Deepfake Detector", layout="wide")
st.sidebar.title("🔍 Navigation")
page = st.sidebar.radio("Go to", ["Home", "Upload & Analyze", "About"])

# Load images
logo1 = Image.open("identity.png")
logo2 = Image.open("man.png")

# Toggle logo
if "logo_toggle" not in st.session_state:
    st.session_state.logo_toggle = False
else:
    st.session_state.logo_toggle = not st.session_state.logo_toggle  # Flip toggle each rerun

# Select logo
selected_logo = logo2 if st.session_state.logo_toggle else logo1
st.sidebar.image(selected_logo, width=50)

# Helper Functions
def extract_frames(video_path, max_frames=MAX_FRAMES):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, FRAME_SIZE)
        frame = frame / 255.0
        frames.append(frame)
        frame_count += 1

    cap.release()
    return np.array(frames)

def extract_audio_mfcc(video_path, max_len=LSTM_INPUT_LENGTH):
    import soundfile as sf
    from pydub import AudioSegment

    audio_path = video_path.replace(".mp4", ".wav")

    # Convert using ffmpeg-compatible method
    sound = AudioSegment.from_file(video_path)
    sound = sound.set_channels(1).set_frame_rate(16000)
    sound.export(audio_path, format="wav")

    y, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=MFCC_FEATURES)
    mfcc = mfcc.T

    if mfcc.shape[0] > max_len:
        mfcc = mfcc[:max_len]
    else:
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

    os.remove(audio_path)
    return np.expand_dims(mfcc, axis=0)

def predict_video_cnn(frames):
    preds = cnn_model.predict(frames)
    avg_pred = np.mean(preds)
    return avg_pred

def predict_audio_lstm(mfcc_features):
    pred = lstm_model.predict(mfcc_features)[0][0]
    return pred

def aggregate_predictions(video_score, audio_score, threshold=THRESHOLD):
    audio_fake = audio_score >= threshold
    video_fake = video_score >= threshold

    if audio_fake and video_fake:
        verdict = "🟥 Fake Audio Fake Video"
    elif audio_fake and not video_fake:
        verdict = "🟨 Fake Audio Real Video"
    elif not audio_fake and video_fake:
        verdict = "🟧 Real Audio Fake Video"
    else:
        verdict = "🟩 Real Audio Real Video"

    return verdict, video_score, audio_score

def save_report(verdict, video_score, audio_score):
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    report = (
        f"Deepfake Detection Report\n"
        f"Date: {now}\n\n"
        f"Final Verdict: {verdict}\n"
        f"Video Score: {video_score:.4f}\n"
        f"Audio Score: {audio_score:.4f}\n"
    )
    path = f"deepfake_report_{now}.txt"
    with open(path, "w") as f:
        f.write(report)
    return path

# Home Page
if page == "Home":
    st.title("🎬 Deepfake Detection System")
    st.markdown("""
    Welcome to the Deepfake Detection System using combined **Audio & Video analysis**.

    This tool allows you to:
    - Upload a video
    - Analyze frames using a CNN (InceptionV3)
    - Analyze audio using an LSTM
    - Combine predictions to detect deepfakes

    Navigate to the **Upload & Analyze** section to begin.
    """)

# Upload & Analyze
elif page == "Upload & Analyze":
    st.title("📤 Upload & Analyze")

    uploaded_file = st.file_uploader("Upload a video file", type=["mp4", "avi", "mov"])

    if uploaded_file is not None:
        st.video(uploaded_file)

        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tmp:
            tmp.write(uploaded_file.read())
            tmp_path = tmp.name

        st.info("Extracting frames and audio...")
        frames = extract_frames(tmp_path)
        mfcc = extract_audio_mfcc(tmp_path)

        st.success(f"Extracted {len(frames)} frames and audio MFCCs.")

        st.info("Running predictions...")
        video_score = predict_video_cnn(frames)
        audio_score = predict_audio_lstm(mfcc)

        verdict, video_score, audio_score = aggregate_predictions(video_score, audio_score)

        st.subheader("🧠 Results")
        st.write(f"**Final Verdict:** {verdict}")
        st.write(f"**Video Fake Score:** {video_score:.2%}")
        st.write(f"**Audio Fake Score:** {audio_score:.2%}")

        # Report Download
        report_path = save_report(verdict, video_score, audio_score)
        with open(report_path, "rb") as file:
            st.download_button("📄 Download Report", data=file, file_name=report_path, mime="text/plain")



# About Page
elif page == "About":
    st.title("ℹ️ About This App")
    st.markdown("""
    This app is built using:
    - **Streamlit** for UI
    - **InceptionV3 CNN** model for video frame analysis trained on OpenForensics Dataset
    - **LSTM model** for audio (MFCC) deepfake detection trained on DeepVoice Dataset

    The dataset used includes real and manipulated videos, where either audio or video content has been altered.
    """)

Overwriting app.py


In [7]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.219.139:8501

your url is: https://early-baboons-play.loca.lt
2025-06-12 16:13:19.843097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749744799.873986   10923 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749744799.883294   10923 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 16:13:26.070953: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
1/1 ━━━━━━━━━━━━━━━━━━━━ 